#### Magic Commands
Magic commands (those that start with `%`) are commands that modify a configuration of Jupyter Notebooks. A number of magic commands are available by default (see list [here](http://ipython.readthedocs.io/en/stable/interactive/magics.html))--and many more can be added with extensions. The magic command added in this section allows `matplotlib` to display our plots directly on the browser instead of having to save them on a local file.

In [1]:
%matplotlib inline

# Activity 8: Re-training a model dynamically
In this activity, we re-train our model every time new data is available.

First, we start by importing `cryptonic`. Cryptonic is a simple software application developed for this course that implements all the steps up to this section using Python classes and modules. Consider Cryptonic a template on how you could develop similar applications.

In [38]:
import cryptonic
import pandas as pd

In [40]:
from tqdm import tqdm_notebook

In [14]:
plt.style.use('seaborn-white')

In [41]:
from cryptonic import Model
from cryptonic import CoinMarketCap
import cryptonic.models.normalizations as normalizations

### Fecthing Real-time Data
Throughout this project we have been using data originally provided by [CoinMarketCap](https://coinmarketcap.com/). We have created an interface for collecting both real-time and historical data as as part of `cryptonic`: the class `CoinMarketCap()`.

In [42]:
print(CoinMarketCap())



        Crypto-currency data comes from the website CoinMarketCap.
        CoinMarketCap is can be accessed at: https://coinmarketcap.com/

        The permission to use the data is available on their FAQ

            https://coinmarketcap.com/faq/

        and reads:

            "Q: Am I allowed to use content (screenshots, data, graphs, etc.) 
            for one of my personal projects and/or commercial use?

            R: Absolutely! Feel free to use any content as you see fit. 
            We kindly ask that you cite us as a source."
        
        


Our model is designed to work with daily data. Let's go ahead and collect historic daily data from CoinMarketCap (this is the same data used previously).

In [43]:
historic_data = CoinMarketCap.historic()

In [44]:
# need to reverse dataframe so that earliest is first
historic_data = historic_data.iloc[::-1].reset_index(drop=True)

In [45]:
historic_data.head(3)

,date,open,high,low,close,volume,market_cap
0,2013-04-28,135.30,135.98,132.10,134.21,None,1500520000
1,2013-04-29,134.44,147.49,134.00,144.54,None,1491160000
2,2013-04-30,144.00,146.93,134.05,139.00,None,1597780000


The data contains practically the same variables from our earlier dataset. However, much of the data comes from an earlier period. Recent Bitcoin prices have gained a lot of volatility if compared to the prices of a few years ago. Before using this data in our model, let's make sure to filter it to dates after January 1, 2017.

In [46]:
#
#  Using the Pandas API, filter the dataframe
#  for observations from 2017 only. 
# 
#  Hint: use the `date` column / variable.
#
model_data = 

### The `Model()` Class
We have also created the class `Model()` which compiles all the code we have written so far. We will use that class to build, train, and evaluate our model.

In [47]:
M = Model(data=model_data,
          variable='close',
          predicted_period_size=7)

In [48]:
M.build()

In [49]:
M.train(epochs=100, verbose=1)

Epoch 1/100
1/1 [==============================] - 2s 2s/step - loss: 0.0023
Epoch 2/100
1/1 [==============================] - 0s 43ms/step - loss: 0.0018
Epoch 3/100
1/1 [==============================] - 0s 38ms/step - loss: 0.0015
Epoch 4/100
1/1 [==============================] - 0s 30ms/step - loss: 0.0013
Epoch 5/100
1/1 [==============================] - 0s 45ms/step - loss: 0.0011
Epoch 6/100
1/1 [==============================] - 0s 41ms/step - loss: 9.3113e-04
Epoch 7/100
1/1 [==============================] - 0s 46ms/step - loss: 8.0604e-04
Epoch 8/100
1/1 [==============================] - 0s 41ms/step - loss: 6.9997e-04
Epoch 9/100
1/1 [==============================] - 0s 42ms/step - loss: 6.0955e-04
Epoch 10/100
1/1 [==============================] - 0s 40ms/step - loss: 5.3154e-04
Epoch 11/100
1/1 [==============================] - 0s 34ms/step - loss: 4.6425e-04
Epoch 12/100
1/1 [==============================] - 0s 59ms/step - loss: 4.0612e-04
Epoch 13/100
1/1 [=====

Epoch 99/100
1/1 [==============================] - 0s 31ms/step - loss: 8.6013e-07
Epoch 100/100
1/1 [==============================] - 0s 32ms/step - loss: 1.0237e-06


We can now use the model for making predictions with the `predict()` method. The parameter `denormalized` will return values in the original scale of the data. In our case, US dollars.

In [50]:
M.predict(denormalized=True)

array([ 6173.45800781,  6402.47314453,  6434.75439453,  6444.95361328,
        6400.10400391,  5788.77832031,  5860.75634766], dtype=float32)

We now evaluate our model to inspect the statistics for the last epoch of training compared to a single test week.

In [51]:
M.evaluate()

{'mape': 3.8799999999999999, 'mse': 0.0, 'rmse': 264.97000000000003}

Finally, we can now save the trained model on disk for later use.

In [52]:
M.save('/bitcoin_model_prod_v0.h5')

Our `Model()` class can also load a previously trained model when instantiated with the `path` parameter.

In [53]:
M = Model(path='/bitcoin_model_prod_v0.h5',
          data=model_data,
          variable='close',
          predicted_period_size=7)

In [54]:
M.predict(denormalized=True)

array([ 6173.45800781,  6402.47314453,  6434.75439453,  6444.95361328,
        6400.10400391,  5788.77832031,  5860.75634766], dtype=float32)

### New Data, Re-train Old Model
One strategy discussed earlier regards the re-training of our model with new data. In our case, our biggest concern is to shape data in a way that the model has been configured. As an example, we will configure our model to predict a week using 40 weeks. We will first train the model with the first 40 weeks of 2017, then continue to re-train it over the following weeks until we reach week 50.

In [55]:
print('Number of full weeks: {}'.format(len(model_data) // 7))

Number of full weeks: 77


First, let's build a model with the first set of data. Notice how we use `7*40 + 7` as the indexer. This is because we use 40 weeks for training and 1 week for testing. 

In [71]:
#######################################
# size of original model_data dataframe
print([len(model_data), len(model_data)/7])
# size of sliced dataframe
print([len(model_data[0:(7*40 + 7 + 1)]), len(model_data[0:(7*40 + 7 + 1)])/7])

[540, 77.14285714285714]
[288, 41.142857142857146]


In [56]:
M = Model(data=model_data[0:(7*40 + 7 + 1)],
          variable='close',
          predicted_period_size=7)

In [57]:
M.build()

In [58]:
M.train()

In [ ]:
#
#  Complete the range function and
#  the model_data filtering parameters
#  using an index to split the data in overlapping
#  groups of 7 days. Then, re-train our model
#  and collect the results.
#
#  The variables A, B, C, and D are placeholders.
#
results = []
for i in range(A, B):
    M.train(model_data[C:D])
    results.append(M.evaluate())

In [60]:
M.predict(denormalized=True)

array([ 5671.36230469,  5897.67041016,  5905.40917969,  5935.04833984,
        6729.01220703,  6977.45166016,  7209.47753906], dtype=float32)

### New Data, New Model
Another strategy is to create and train a new model evey time new data is available. This approach tends to reduce catastrophic forgetting, but training time increases as data increases. 

It's implementation is quite simple.

Let's assume we have old data for 49 weeks of 2017 and after a week we now have new data. We represent this wtih the variables `old_data` and `new_data`. 

In [61]:
old_data = model_data[0*7:7*48 + 7 + 1]

In [62]:
new_data = model_data[0*7:7*49 + 7 + 1]

In [63]:
M = Model(data=old_data,
          variable='close',
          predicted_period_size=7)

In [64]:
M.build()
M.train()

In [65]:
M.predict(denormalized=True)

array([ 15498.28125   ,  15999.93164062,  16182.19238281,  19424.1953125 ,
        24333.51757812,  22497.48242188,  20619.40234375], dtype=float32)

Now, assume that new data is available. Using this technicle we go ahead and create a new model using only the new data. 

In [66]:
#
#  Re-instantiate the model with the Model()
#  class using the new_data variable instead
#  of the old_data one. 
#

In [67]:
M.build()
M.train()

In [68]:
M.predict(denormalized=True)

array([ 15530.66503906,  15826.07324219,  16193.54296875,  19562.61328125,
        24507.19335938,  22695.15625   ,  20839.10546875], dtype=float32)

This approach is very simple to implement and tends to work well. We will be using this to deploy our application.